# Продвинутый анализ данных для юристов

## Обработка текстов

### Токенизация, лемматизация, стемминг, стоп-слова. Векторизация (TF-IDF), Word2Vec

#### Матвей Бакшук, преподаватель ФКН НИУ ВШЭ


Материал подготовлен с использованием [статьи на хабр](https://habr.com/ru/companies/otus/articles/755772/)

<p align = 'justify'> Изучение текстовых данных является одной из фундаментальных задач в области анализа данных и машинного обучения. Однако тексты представляют собой сложные и многомерные структуры, которые не могут быть напрямую обработаны алгоритмами машинного обучения. В этом контексте извлечение признаков — это процесс преобразования текстовых данных в числовые векторы, которые могут быть использованы для обучения моделей и анализа. Этот шаг играет ключевую роль в предварительной обработке данных перед применением алгоритмов. </p>

<p align = 'justify'><code>Term Frequency-Inverse Document Frequency (TF-IDF)</code> — это один из наиболее распространенных и мощных методов для извлечения признаков из текстовых данных. <code>TF-IDF</code> вычисляет важность каждого слова в документе относительно количества его употреблений в данном документе и во всей коллекции текстов. Этот метод позволяет выделить ключевые слова и понять, какие слова имеют больший вес для определенного документа в контексте всей коллекции. </p>

Для начала давайте разберёмся с используемыми понятиями и посмотрим на простую формулу

### Термины `TF (Term Frequency)` и `IDF (Inverse Document Frequency)`

<ul>
<li><b>TF (Частота термина)</b> обозначает, насколько часто определенное слово появляется в данном документе. Таким образом, TF измеряет важность слова в контексте отдельного документа.</li>

<li><b> IDF (Обратная частота документа)</b> измеряет, насколько уникально слово является по всей коллекции документов. Слова, которые появляются в большинстве документов, имеют низкое IDF, так как они не вносят большой информационной ценности.</li>
</ul>

Формула `TF-IDF` комбинирует понятия `TF` и `IDF`, чтобы вычислить важность каждого слова в каждом документе. Формально, формула выглядит следующим образом:

$$TF\text{-}IDF(t, d) = TF(t, d) * IDF(t)$$

где:

$TF(t, d)$ - Частота термина $(TF)$ для слова $t$ в документе $d$

$IDF(t)$ - Обратная частота документа $(IDF)$ для слова $t$

<p align = 'justify'>Давайте рассмотрим простой корпус текстов и на его примере посмотрим на применение методов <code>TF-IDF</code>. Возьмем такие маленькие тексты:</p>

<ol>
<li>'Машинное обучение — это интересная область.'</li>

<li>'Обучение с учителем — ключевой аспект машинного обучения.'</li>

<li>'Область NLP также связана с машинным обучением.'</li>
</ol>

<p align = 'justify'>Перед тем как вычислять <code>TF-IDF</code>, мы должны выполнить предварительную обработку, такую как удаление стоп-слов, приведение к нижнему регистру и <b>токенизация</b> — разбиение текстов на отдельные слова или токены.</p>

### Расчет TF-IDF для слов в корпусе
Представим, что мы хотим вычислить `TF-IDF` для слова "машинное" в первом документе. Давайте предположим, что `TF` для этого слова равен 1 (поскольку оно встречается 1 раз в данном документе), а `IDF` можно вычислить как общее количество документов (3) деленное на количество документов, в которых встречается это слово (2). Таким образом, IDF для слова "машинное" равен $log(\frac{3}{2}) = 0.18$ (про появление здесь логарифма поговорим в следующем блоке)

Теперь мы можем вычислить `TF-IDF` для слова "машинное" в первом документе: $$TF-IDF = 1 * 0.18 = 0.18$$
Продолжая этот процесс для каждого слова в каждом документе, мы можем создать матрицу `TF-IDF`, где строки представляют слова, а столбцы - документы.

### Логарифм в формуле IDF (Inverse Document Frequency)

1. **Сглаживание весов**:
   - Без логарифма `IDF` вычислялся бы как $ \dfrac{N}{df_t} $, где $ N $ — общее число документов, а $ df_t $ — число документов, содержащих слово $ t $.
   - Это значение может быть очень большим для редких слов (например, если слово встречается в 1 документе из 1000, `IDF` = 1000). Логарифм смягчает эту разницу, делая веса более управляемыми.

2. **Учет "информативности"**:
   - Логарифм отражает идею, что разница между 1 и 10 документами (где слово встречается) гораздо значимее, чем между 100 и 110. Это соответствует принципу уменьшающейся полезности (diminishing returns).

3. **Математическая устойчивость**:
   - Логарифм предотвращает доминирование редких слов с очень высоким IDF над частыми словами. Без него редкие термины могли бы искусственно завышать вес в итоговом TF-IDF.

### Формула IDF:
Обычно используется один из вариантов:

$$\text{IDF} = \log\left(\frac{N}{df_t}\right) \quad \text{или} \quad \log\left(\frac{N}{df_t} + 1\right)$$
где $ N $ — общее число документов, $ df_t $ — число документов со словом $ t $ .

Логарифмическая шкала лучше соответствует человеческому восприятию "важности". Например:
- Если слово есть во всех документах ($df_t = N$), `IDF` становится $\log(1) = 0$ — такое слово не несет полезной информации для различения документов.
- Редкие слова получают умеренно высокий вес, а не экстремальный.

Это делает `TF-IDF` более устойчивым и интерпретируемым.

### Преимущества и ограничения TF-IDF

Преимущества использования `TF-IDF` для извлечения признаков
`TF-IDF` предоставляет несколько ключевых преимуществ:

1. **Учет важности слов:** `TF-IDF` учитывает как частоту слова в документе, так и его общую редкость по всей коллекции. Таким образом, он помогает выделять ключевые слова, которые часто встречаются в данном документе, но не слишком распространены в остальных.

2. **Устранение шума:** Слова, которые встречаются в большинстве документов (стоп-слова), имеют низкий `IDF` и, следовательно, низкий общий вес `TF-IDF`. Это позволяет устранить шум и фокусироваться на более важных словах.

### Ограничения метода и ситуации, в которых он может быть неэффективен

1. **Отсутствие семантической информации:** `TF-IDF` не учитывает семантические связи между словами, что может привести к ограниченной способности понимания смысла текста.

2. **Чувствительность к длине документа:** Длинные документы могут иметь более высокие значения `TF`, даже если ключевые слова встречаются реже. В таких случаях, `TF-IDF` может недооценить важность конкретных слов.

Важно понимать, в каких ситуациях `TF-IDF` будет эффективен, а когда стоит рассмотреть альтернативные методы.

### Еще один простой пример:



![text](https://github.com/M4tthew27/DATA/blob/main/tfidf.png?raw=true)

### Применение TF-IDF в задачах анализа текстов

#### A. Извлечение ключевых слов и терминов

Извлечение ключевых слов из текстовых данных является одним из наиболее распространенных сценариев использования `TF-IDF`. Одним из способов сделать это - это выбрать топ-N слов с наибольшими значениями `TF-IDF`. Давайте рассмотрим пример с кодом:

In [17]:
#pip install sklearn или !pip install sklearn
# pip install --upgrade numpy scikit-learn scipy

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Пример текстовых данных
documents = [
    "Машинное обучение - это интересная область.",
    "Обучение с учителем - ключевой аспект машинного обучения.",
    "Область NLP также связана с машинным обучением."
]

# Создание объекта TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Применение TF-IDF к текстовым данным
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

print(f'Формат изначальной матрицы:\n{tfidf_matrix}') # формат "разреженной матрицы: большинство значений нули."
# i - документ, j - термин (слово)

# Получаем список терминов (слов)
feature_names = tfidf_vectorizer.get_feature_names_out()

In [ ]:
# Для примера, давайте посмотрим на получение списка ключевых слов и их значения TF-IDF для первого документа

tfidf_scores = tfidf_matrix.toarray()[0] # tfidf_matrix.toarray() позволяет отразить нулевые значения, [0] — первый документ

# Посмотрим на два объекта, которые у нас получились:
print(f'Результаты: {tfidf_scores}')
print(f'Слова: {feature_names}')

# Сортировка слов по значениям TF-IDF
sorted_keywords = [word for i, word in sorted(zip(tfidf_scores, feature_names), reverse=True)]

print("Ключевые слова из корпуса текстов в отсортированном порядке", sorted_keywords)

### Применение ключевых слов в поисковых системах
Извлечение ключевых слов с помощью `TF-IDF` позволяет улучшить поиск и индексацию текстовых данных. Представьте, что вы хотите построить поисковую систему для коллекции документов. Вы можете извлечь ключевые слова для каждого документа с помощью `TF-IDF` и использовать их для индексации и ранжирования результатов поиска.


### B. Кластеризация и категоризация текстовых данных

#### Группировка текстов по сходству на основе `TF-IDF`

`TF-IDF` также может быть использован для кластеризации текстовых данных, то есть группировки схожих документов в один кластер. Кластеризация может помочь выявить общие темы и понять структуру данных. Рассмотрим следующий пример:

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Применение кластеризации KMeans к матрице TF-IDF

num_clusters = 2 # установим 2, поскольку у нас всего три текста
kmeans = KMeans(n_clusters=num_clusters, random_state=0) 

# Кратко про метод k-средних: предоставляет разбиение набора данных на k кластеров, 
# таких, что каждый объект принадлежит кластеру с ближайшим центром кластера. Иными словами, мы ищем 
# скопление близких наблюдений в данных, чтобы добиться деления на группы (в данном примере у нас это)
# группы текстов.

# Цель алгоритма — минимизировать суммарное расстояние точек кластеров от их центров (частно: выделить схожие между
# собой документы.

## Заполняем на основании нашей матрицы:

kmeans.fit(tfidf_matrix)

# Чтобы показать примеры документов в каждом кластере

for cluster_id in range(num_clusters):
    cluster_indices = np.where(kmeans.labels_ == cluster_id)[0]
    print(f"Кластер {cluster_id + 1}:")
    for idx in cluster_indices:
        print(documents[idx])
    print("--------")

На пример трёх документов это просто и может казаться очевидным, однако на реальной практике данных больше, и число кластеров также может быть настроено

Практические примеры применения кластеризации

+ **Новостные порталы**: новостные статьи могут быть автоматически категоризированы по темам с использованием кластеризации на основе TF-IDF.

+ **Социальные сети**: кластеризация сообщений или постов пользователей может помочь создать персонализированные ленты новостей.

##### Юриспруденция: 

+ **Автоматическая категоризация правовых документов**: кластеризация судебных решений, договоров или нормативных актов по темам (например: "налоговые споры", "трудовые конфликты", "арбитражные дела").

+ **Анализ судебной практики**: группировка схожих судебных решений по ключевым параметрам (статьям закона, сумме иска, категории ответчика). Например, выявление кластеров дел, по которым чаще всего выносятся оправдательные приговоры. 

И т.д.

### C. Классификация текстов

##### Обучение модели классификации на основе `TF-IDF`

`TF-IDF` также может быть использован для классификации текстов на основе их содержания. Для этого мы можем обучить модель машинного обучения на векторах `TF-IDF`, представляющих документы, и затем использовать эту модель для предсказания категории новых текстов. Пример:



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Пример данных для классификации
categories = ["технологии", "спорт"]
data = [
    # Технологии
    "Новая модель смартфона вышла на рынок с улучшенной камерой.",
    "Разработка искусственного интеллекта активно продвигается.",
    "Компания представила инновационный чип для ноутбуков.",
    "Учёные создали робота, способного выполнять хирургические операции.",
    "Вышло обновление операционной системы с новыми функциями.",
    "Разработан квантовый процессор нового поколения.",
    "Инженеры тестируют дроны для доставки товаров.",
    "Крупная ИТ-компания инвестирует в развитие облачных сервисов.",
    "Появился новый язык программирования для веб-разработки.",
    "Ведется разработка гибких дисплеев для смартфонов.",
    # Спорт
    "Сборная России победила в международном турнире по футболу.",
    "Олимпийские игры пройдут в следующем году в Париже.",
    "Теннисист выиграл престижный турнир в Лондоне.",
    "Формула-1 представила обновлённый регламент гонок.",
    "Хоккейная команда одержала уверенную победу в плей-офф.",
    "Боксёр нокаутировал соперника в первом раунде.",
    "Соревнования по легкой атлетике прошли в Москве.",
    "Футболист перешёл в новый клуб за рекордную сумму.",
    "Прошел финал чемпионата мира по баскетболу.",
    "Пловец установил новый мировой рекорд."
]

labels = [0]*10 + [1]*10 # ставим 0 для технологий и 1 для спорта

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=0, stratify=labels) 
# делим 


# Преобразование текстовых данных в матрицу TF-IDF

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train) # обучаем векторизатор
X_test_tfidf = tfidf_vectorizer.transform(X_test) # просто трансформируем в матрицу, тк предсказывать будем по обученной модели

# Обучение модели классификации
classifier = MultinomialNB() # берем просто для примера, тк позволяет работать с дискретными признаками
classifier.fit(X_train_tfidf, y_train)

# Предсказание категорий для тестовых данных
y_pred = classifier.predict(X_test_tfidf)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

### Сопоставление TF-IDF с Word2Vec, Doc2Vec и другими методами

Существует множество других методов извлечения признаков из текста, таких как `Word2Vec`, `Doc2Vec`, `FastText` и многие другие. Эти методы, в отличие от `TF-IDF`, учитывают семантические связи между словами и векторное представление слов. Поэтому для некоторых задач, особенно связанных с семантическим анализом, они могут быть более эффективными.

#### Выбор подходящего метода в зависимости от задачи

Выбор метода извлечения признаков зависит от конкретной задачи и характеристик текстовых данных. `TF-IDF` подходит хорошо для задач, связанных с извлечением ключевых слов, кластеризацией и классификацией. Однако для более сложных задач, где важны семантические отношения между словами, стоит рассмотреть использование более современных методов, таких как `Word2Vec`.

### Практические советы по использованию TF-IDF
#### Подготовка и предобработка текстовых данных
<b>Удаление стоп-слов и специальных символов</b>

Перед применением `TF-IDF` к текстовым данным, часто полезно провести предварительную обработку данных. Одним из широко распространенных шагов является удаление стоп-слов - слов, которые не несут смысловой нагрузки (например, предлоги, союзы) и специальных символов. Для этого можно использовать библиотеку Natural Language Toolkit `(NLTK)` на Python:

In [15]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Загрузка стоп-слов и пунктуации
nltk.download('stopwords') # найдем их локально чтобы посмотреть, что за файл у нас скачался!
nltk.download('punkt')
stop_words = set(stopwords.words('russian'))
punctuation = set(string.punctuation)

def preprocess_text(text):
    words = word_tokenize(text.lower())  # Привести к нижнему регистру и токенизировать
    filtered_words = [word for word in words if word not in stop_words and word not in punctuation]
    return " ".join(filtered_words)

# Пример предобработки текстовых данных
text = "Машинное обучение - это интересная область, изучаемая многими."
preprocessed_text = preprocess_text(text)
print("Предобработанный текст:", preprocessed_text)

Предобработанный текст: машинное обучение это интересная область изучаемая многими


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matveybaksuk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matveybaksuk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Приведение к нижнему регистру и лемматизация

Кроме удаления стоп-слов, также полезно привести текст к нижнему регистру и выполнить лемматизацию - приведение слов к их базовой форме. Это позволяет уменьшить разнообразие форм слова и улучшить качество извлекаемых признаков. Воспользуемся библиотекой `spaCy` для лемматизации:



In [14]:
import spacy 
# Загрузка языковой модели spaCy
nlp = spacy.load("ru_core_news_sm")

def preprocess_and_lemmatize(text):
    doc = nlp(text.lower())  # Привести к нижнему регистру и лемматизировать
    lemmatized_words = [token.lemma_ for token in doc if token.text not in punctuation and token.text not in stop_words]
    return " ".join(lemmatized_words)

# Пример предобработки и лемматизации текста
text = "Машинное обучение - это интересная область, изучаемая многими."
preprocessed_lemmatized_text = preprocess_and_lemmatize(text)
print("Предобработанный и лемматизированный текст:", preprocessed_lemmatized_text)

Предобработанный и лемматизированный текст: машинный обучение это интересный область изучаемая многими


## Заключение

Использование `TF-IDF` открывает перед нами широкий спектр возможностей для анализа и интерпретации текстовых данных. Совместно с современными методами и инструментами анализа, он помогает в создании информативных моделей и понимании семантических связей в текстах. При использовании правильных методов предобработки и параметров `TF-IDF`, вы сможете добиться высокой точности и эффективности в анализе текстовых данных

Также, существуют и другие, более сложные методы для работы с текстами (например, статья про [word2vec](https://habr.com/ru/articles/801807/) описывает одну из самых популярных моделей для этого)